In [1]:
#IMPORTING STUFFS
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import numpy as np
from sklearn.utils import shuffle
import pandas as pd 
from skimage.io import imread, imshow
from skimage.transform import rescale
from os import listdir
from sklearn.model_selection import train_test_split as tts

In [2]:
class My_Custom_Generator(keras.utils.Sequence) :
  
  def __init__(self, image_filenames, labels, batch_size) :
    self.image_filenames = image_filenames
    self.labels = labels
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
    return np.array(batch_x), np.array(batch_y)

In [3]:
#GETTING DATA
X = []
Y = []
for filename in listdir('/content/drive/MyDrive/WBC_data_kaggle/main_dataset/test/EOSINOPHIL'):
    # load image
    img_data = imread('/content/drive/MyDrive/WBC_data_kaggle/main_dataset/test/EOSINOPHIL/' + filename,as_gray=True)
    #Rescaling the image
    image_r = rescale(img_data, 0.5, anti_aliasing=False)
    X.append(image_r)
    Y.append(0)

for filename in listdir('/content/drive/MyDrive/WBC_data_kaggle/main_dataset/test/LYMPHOCYTE'):
    # load image
    img_data = imread('/content/drive/MyDrive/WBC_data_kaggle/main_dataset/test/LYMPHOCYTE/' + filename,as_gray=True)
    #Rescaling the image
    image_r = rescale(img_data, 0.5, anti_aliasing=False)
    X.append(image_r)
    Y.append(1)

for filename in listdir('/content/drive/MyDrive/WBC_data_kaggle/main_dataset/test/MONOCYTE'):
    # load image
    img_data = imread('/content/drive/MyDrive/WBC_data_kaggle/main_dataset/test/MONOCYTE/' + filename,as_gray=True)
    #Rescaling the image
    image_r = rescale(img_data, 0.5, anti_aliasing=False)
    X.append(image_r)
    Y.append(2)

for filename in listdir('/content/drive/MyDrive/WBC_data_kaggle/main_dataset/test/NEUTROPHIL'):
    # load image
    img_data = imread('/content/drive/MyDrive/WBC_data_kaggle/main_dataset/test/NEUTROPHIL/' + filename,as_gray=True)
    #Rescaling the image
    image_r = rescale(img_data, 0.5, anti_aliasing=False)
    X.append(image_r)
    Y.append(3)

X_arr = np.array(X)
Y_arr = np.array(Y)

X_fi,Y_fi = shuffle(X_arr,Y_arr)

In [4]:
print(X_fi.shape)
print(Y_fi.shape)

(600, 120, 160)
(600,)


In [5]:
X_train, X_test, y_train, y_test = tts(X_fi, Y_fi, test_size=0.33)

In [11]:
#RESHAPING TO PASS THE ARRAY TO CNN

X_train = X_train.reshape(402,120,160,1)
X_train.shape
# plt.matshow(X_train[0])


batch_size = 4

my_training_batch_generator = My_Custom_Generator(X_train, y_train, batch_size)
my_validation_batch_generator = My_Custom_Generator(X_test,y_test, batch_size)

In [14]:
#MODEL
model = keras.Sequential([
    #cnn layers
    keras.layers.Conv2D(filters=32,kernel_size=(3, 3),activation='relu',input_shape=(120,160,1)),
    keras.layers.MaxPooling2D((2,2)),

    keras.layers.Conv2D(filters=64,kernel_size=(3, 3),activation='relu'),
    keras.layers.MaxPooling2D((2,2)),

    #dense normal ann
    keras.layers.Flatten(),     
    
    keras.layers.Dense(60,activation='relu'),
    
    keras.layers.Dense(4,activation='softmax'),
    #softmax normalises the probability of the output set
])


model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy']
)

#model.fit(X_train,y_train,epochs=20)
model.fit_generator(generator=my_training_batch_generator, steps_per_epoch = int(402 // batch_size), epochs = 10, verbose = 1,
                    validation_data = my_validation_batch_generator, validation_steps = int(198 // batch_size))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
100/100 [==============================] - 2s 17ms/step - loss: 1.5152 - accuracy: 0.2487 - val_loss: 1.3865 - val_accuracy: 0.2602
Epoch 2/10
100/100 [==============================] - 1s 13ms/step - loss: 1.3812 - accuracy: 0.3141 - val_loss: 1.3883 - val_accuracy: 0.2347
Epoch 3/10
100/100 [==============================] - 1s 13ms/step - loss: 1.3470 - accuracy: 0.3794 - val_loss: 1.4163 - val_accuracy: 0.2500
Epoch 4/10
100/100 [==============================] - 1s 13ms/step - loss: 1.2485 - accuracy: 0.4648 - val_loss: 1.4755 - val_accuracy: 0.2653
Epoch 5/10
100/100 [==============================] - 1s 13ms/step - loss: 1.0841 - accuracy: 0.5503 - val_loss: 1.5759 - val_accuracy: 0.2653
Epoch 6/10
100/100 [==============================] - 1s 13ms/step - loss: 0.8774 - accuracy: 0.6307 - val_loss: 1.7851 - val_accuracy: 0.2500
Epoch 7/10
100/100 [==============================] - 1s 14ms/step - loss: 0.6557 - accuracy: 0.7688 - val_loss: 2.3541 - val_accuracy: 0.2806

In [13]:
#EVALUATION
X_test = X_test.reshape(198,120,160,1)
model.evaluate(X_test,y_test)

7/7 [==============================] - 0s 12ms/step - loss: 2.7062 - accuracy: 0.1970


[2.7062370777130127, 0.19696970283985138]